In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 23)
arquivo = pd.read_csv('recipeData.csv', encoding='ISO-8859-1')

In [ ]:
selecao = arquivo.loc[arquivo['StyleID'].isin([7, 10, 134, 9, 4, 30, 86, 12, 92, 6, 175, 39])]

In [ ]:
selecao.drop(['BeerID', 'Name', 'URL', 'Style', 'UserId', 'PrimingMethod', 'PrimingAmount'], axis=1)

In [ ]:
selecao.loc[selecao['SugarScale'] == 'Specific Gravity', 'SugarScale'] = 0
selecao.loc[selecao['SugarScale'] == 'Plato', 'SugarScale'] = 1

In [ ]:
# transformando variáveis texto na coluna 'BrewMethod' em categorias com one hot encoding
brewmethod_encode = pd.get_dummies(selecao['BrewMethod'])

# excluindo a coluna de texto 'BrewMethod'
selecao = selecao.drop('BrewMethod', axis=1)

# inserindo as variáveis one hot encode novamente no dataset
df = pd.concat([selecao, brewmethod_encode], axis=1)

In [ ]:
df.head(15)

In [ ]:
faltantes = df.isnull().sum()
faltantesPercentual = (faltantes/ len(df['StyleID'])) * 100

print(faltantesPercentual)

In [ ]:
df['BoilGravity'].fillna(df['BoilGravity'].median(), inplace=True)

In [ ]:
x_treino = df[df['PitchRate'].notnull()]
x_treino.drop('PitchRate', axis=1, inplace=True)
y_treino = df[df['PitchRate'].notnull()]['PitchRate']
x_preench = df[df['PitchRate'].isnull()]
x_preench.drop('PitchRate', axis=1, inplace=True)
y_preench = df[df['PitchRate'].isnull()]['PitchRate']

In [ ]:
x_treino = x_treino.drop(['MashThickness', 'PrimaryTemp'], axis=1)
x_preench = x_preench.drop(['MashThickness', 'PrimaryTemp'], axis=1)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# aplicando o modelo
modelo = DecisionTreeRegressor()
modelo.fit(x_treino, y_treino)

# predição dos valores
y_preech = modelo.predict(x_preench)

In [ ]:
df = df.PitchRate[df.PitchRate.isnull()] = y_preench